In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#!pip install ultralytics==8.3.138
#!pip install tensorflow==2.12.1
#!pip install typing_extensions==4.13.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.155
    Uninstalling ultralytics-8.3.155:
      Successfully uninstalled ultralytics-8.3.155
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.14.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.12.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 whic

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 129.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.

In [ ]:
#working demo using gradio

In [4]:
import gradio as gr
import cv2
from ultralytics import YOLO
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import smtplib
from email.mime.text import MIMEText
import time
import os # For environment variables

# --- Email Configuration (RECOMMENDED: Use environment variables for sensitive info) ---
# Set these in your environment, e.g., in a Hugging Face Space Secrets
EMAIL_ADDRESS = 'vidhyachandran2007@gmail.com'
EMAIL_PASSWORD = 'uqvl mzad ajfj famb'  # Your email app password
RECEIVER_EMAIL = 'vidyajith2010@gmail.com'
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587

ALERT_SUBJECT = "Security Alert: Person Consistently Detected!"
ALERT_BODY_PREFIX = "A person has been consistently detected and identified in the zone for several seconds: "

# --- Load Models (Load once outside the processing loop for efficiency) ---
yolo_model = YOLO("yolov8n.pt") # YOLOv8 nano model

# Placeholder for your Keras model and labels.
# In a Gradio demo, these should be accessible on the server where the demo runs.
# Make sure your 'keras_model.h5' and 'labels.txt' are in the same directory
# as your app.py or a path accessible to it.
try:
    classifier_model = load_model("/content/drive/MyDrive/YOLO_TM/keras_model.h5", compile=False)
    with open("/content/drive/MyDrive/YOLO_TM/labels.txt", "r", encoding="utf-8") as f:
        class_names = [line.strip() for line in f.readlines()]
    print("Keras model and labels loaded successfully.")
except Exception as e:
    print(f"Error loading Keras model or labels: {e}")
    print("Please ensure 'keras_model.h5' and 'labels.txt' are in the correct path.")
    classifier_model = None # Handle cases where model loading fails
    class_names = []

input_size = (224, 224) # Input size for your Keras classifier

# --- Persistent Detection Configuration ---
CONSECUTIVE_DETECTION_DURATION = 5 # Seconds for continuous detection to trigger an alert
EMAIL_COOLDOWN_SECONDS = 300 # Send an email no more than every 5 minutes

# --- Email Sending Function ---
def send_alert_email(classification_details, last_email_time_state):
    # The global `LAST_EMAIL_SENT_TIME` is replaced by `last_email_time_state` here
    if time.time() - last_email_time_state < EMAIL_COOLDOWN_SECONDS:
        print(
            f"Email alert cooldown active. Next email allowed in {EMAIL_COOLDOWN_SECONDS - (time.time() - last_email_time_state):.0f} seconds.")
        return last_email_time_state # Return unchanged state

    msg = MIMEText(ALERT_BODY_PREFIX + classification_details)
    msg['Subject'] = ALERT_SUBJECT
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = RECEIVER_EMAIL

    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.send_message(msg)
        print("Persistent alert email sent successfully!")
        return time.time() # Update and return new timestamp
    except Exception as e:
        print(f"Error sending email: {e}")
        return last_email_time_state # Return unchanged state on error

# --- Main Processing Function for Gradio ---
# This function will be called for each frame from the webcam or video.
def process_frame(frame, detection_start_time_state, last_email_time_state):
    if frame is None:
        # Gradio sends `None` when the webcam is first starting or has issues.
        # It also sends `None` if the component is reset/cleared.
        # Return an empty image and the current state to avoid errors.
        empty_image = np.zeros((480, 640, 3), dtype=np.uint8) # Or adjust to your expected frame size
        return empty_image, detection_start_time_state, last_email_time_state # Return current state

    # Gradio `gr.Image` with `sources="webcam"` provides RGB NumPy arrays.
    # OpenCV typically expects BGR, so convert if you're using OpenCV drawing functions.
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    current_frame_alert_condition_met = False
    display_alert_text = False
    display_info_list = []

    # YOLO Inference
    yolo_results = yolo_model(frame, verbose=False)

    for yolo_result in yolo_results:
        yolo_boxes = yolo_result.boxes
        for yolo_box in yolo_boxes:
            x1_yolo, y1_yolo, x2_yolo, y2_yolo = map(int, yolo_box.xyxy[0])
            label_yolo = yolo_model.names[int(yolo_box.cls[0])]

            if label_yolo.lower() == 'person':
                temp_display_label = label_yolo
                temp_box_color = (0, 255, 0) # Green by default

                person_crop = frame[y1_yolo:y2_yolo, x1_yolo:x2_yolo]
                if person_crop.size == 0 or person_crop.shape[0] == 0 or person_crop.shape[1] == 0:
                    continue # Skip empty crops

                if classifier_model is not None and class_names:
                    try:
                        # Ensure crop is not empty or malformed before processing
                        if person_crop.shape[0] > 0 and person_crop.shape[1] > 0:
                            rgb_crop = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)
                            pil_image = Image.fromarray(rgb_crop)
                            pil_image = ImageOps.fit(pil_image, input_size, Image.Resampling.LANCZOS)

                            image_array = np.asarray(pil_image).astype(np.float32)
                            normalized = (image_array / 127.5) - 1.0
                            data = np.expand_dims(normalized, axis=0)

                            prediction = classifier_model.predict(data, verbose=0)
                            idx = np.argmax(prediction)
                            cls_name = class_names[idx][2:] # Remove '0 ' or '1 ' prefix if present
                            cls_conf = prediction[0][idx]

                            temp_display_label = f"{cls_name} ({cls_conf:.2f})"

                            if cls_name.lower() == 'others' and cls_conf > 0.7:
                                current_frame_alert_condition_met = True
                                temp_box_color = (0, 255, 255) # Yellow
                                temp_display_label = f"Potential ALERT! {temp_display_label}"
                        else:
                            temp_display_label = "Invalid crop"
                            temp_box_color = (0, 0, 0) # Black for invalid
                    except Exception as e:
                        print(f"Classifier error on frame: {e}")
                        temp_display_label = "Unknown (Classifier Error)"
                        temp_box_color = (0, 165, 255) # Orange
                else:
                    temp_display_label = "Classifier not loaded"
                    temp_box_color = (128, 128, 128) # Gray

                display_info_list.append({
                    'coords': (x1_yolo, y1_yolo, x2_yolo, y2_yolo),
                    'label': temp_display_label,
                    'color': temp_box_color
                })

    # --- Persistent Detection Logic (State Management) ---
    new_detection_start_time = detection_start_time_state
    new_last_email_time = last_email_time_state

    if current_frame_alert_condition_met:
        if new_detection_start_time is None:
            new_detection_start_time = time.time()
            print(f"Alert condition first met. Starting {CONSECUTIVE_DETECTION_DURATION}s countdown...")
        else:
            elapsed_time = time.time() - new_detection_start_time
            if elapsed_time >= CONSECUTIVE_DETECTION_DURATION:
                # Trigger email and update `new_last_email_time`
                new_last_email_time = send_alert_email(
                    f"Unknown person detected persistently for {CONSECUTIVE_DETECTION_DURATION:.1f} seconds.",
                    new_last_email_time
                )
                display_alert_text = True
    else:
        if new_detection_start_time is not None:
            print("Alert condition no longer met. Resetting countdown.")
        new_detection_start_time = None
        display_alert_text = False

    # --- Draw all stored detections ---
    for info in display_info_list:
        x1, y1, x2, y2 = info['coords']
        label = info['label']
        color = info['color']

        if display_alert_text and "ALERT!" in label:
            color = (0, 0, 255) # Red for active alert

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # --- Display global countdown/alert status ---
    if new_detection_start_time is not None:
        elapsed_time = time.time() - new_detection_start_time
        countdown_text = f"Persistent Detect: {elapsed_time:.1f}/{CONSECUTIVE_DETECTION_DURATION}s"
        cv2.putText(frame, countdown_text, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    if display_alert_text and new_detection_start_time is not None and (
                time.time() - new_detection_start_time) >= CONSECUTIVE_DETECTION_DURATION:
        cv2.putText(frame, "EMAIL SENT (COOLDOWN)", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Convert back to RGB for Gradio display
    processed_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Return the processed frame and updated state variables
    return processed_frame, new_detection_start_time, new_last_email_time

# --- Gradio Interface ---
with gr.Blocks() as demo:
    gr.Markdown("# Live Security Alert System")
    gr.Markdown(
        "This demo uses YOLOv8 for person detection and a Keras classifier "
        "to identify 'others'. An email alert is sent if 'others' are "
        f"consistently detected for {CONSECUTIVE_DETECTION_DURATION} seconds."
    )

    with gr.Row():
        # Corrected input for live video (webcam)
        live_video_input = gr.Image(sources=["webcam"], streaming=True, label="Live Webcam Feed")
        # Output for processed video
        output_video = gr.Image(label="Processed Output (Live)", streaming=True) # Use Image for faster display per frame

    # State variables to maintain context across frames
    detection_start_time_state = gr.State(None)
    last_email_time_state = gr.State(0.0) # Initialize to float 0.0 so first email can send

    # Link input to output via the processing function
    # Use .stream() for continuous webcam input processing
    live_video_input.stream(
        fn=process_frame,
        inputs=[live_video_input, detection_start_time_state, last_email_time_state],
        outputs=[output_video, detection_start_time_state, last_email_time_state]
    )


    gr.Markdown("## How to Use:")
    gr.Markdown(
        "1. Allow webcam access when prompted.\n"
        "2. The live feed will start. Your system will process frames.\n"
        "3. Watch the 'Processed Output (Live)' for detections and alerts.\n"
        "4. Email alerts will be sent if conditions are met (check console for logs)."
    )
    gr.Markdown("---")
    gr.Markdown("### Important: Email Configuration")
    gr.Markdown(
        "For the email alerts to work, you need to set environment variables "
        "`EMAIL_ADDRESS`, `EMAIL_APP_PASSWORD`, and `RECEIVER_EMAIL` "
        "(e.g., in Hugging Face Spaces secrets). **Do NOT hardcode your main password!**"
    )

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch(debug=True) # debug=True shows more logs in console

Keras model and labels loaded successfully.
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://88d51b7511ed53020f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Alert condition first met. Starting 5s countdown...
Alert condition no longer met. Resetting countdown.
Alert condition first met. Starting 5s countdown...
Persistent alert email sent successfully!
Alert condition no longer met. Resetting countdown.
Alert condition first met. Starting 5s countdown...
Alert condition no longer met. Resetting countdown.
Alert condition first met. Starting 5s countdown...
Alert condition no longer met. Resetting countdown.
Alert condition first met. Starting 5s countdown...
Alert condition no longer met. Resetting countdown.
Alert condition first met. Starting 5s countdown...
Alert condition no longer met. Resetting countdown.
Alert condition first met. Starting 5s countdown...
Alert condition no longer met. Resetting countdown.
Alert condition first met. Starting 5s countdown...
Alert condition no longer met. Resetting countdown.
Alert condition first met. Starting 5s countdown...
Alert condition no longer met. Resetting countdown.
Alert condition first 